In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df=df.dropna()

In [6]:
x=df.drop('label',axis=1)

In [7]:
 y=df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [8]:
import tensorflow as tf
tf.__version__

'2.6.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
voc_size=5000

In [11]:
messages=x.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [15]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [16]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3999, 3131, 225, 2619, 1020, 374, 1480, 791, 3599, 1269],
 [1651, 4694, 3720, 3191, 1661, 1731, 4006],
 [3456, 3704, 528, 1966],
 [478, 2708, 2539, 4898, 935, 3495],
 [4383, 1661, 3353, 2529, 3412, 1265, 1661, 3250, 4235, 806],
 [2927,
  325,
  1635,
  4759,
  3981,
  4551,
  1343,
  2432,
  2585,
  3813,
  3441,
  4445,
  3259,
  563,
  4006],
 [4447, 3502, 4606, 2349, 2428, 2991, 312, 324, 3020, 3882, 3200],
 [3720, 4015, 1856, 2506, 4269, 2019, 4551, 2441, 3020, 3882, 3200],
 [2911, 4696, 2382, 1858, 2255, 3347, 3901, 1345, 4551, 2861],
 [3716, 4503, 4884, 2881, 773, 123, 1817, 173],
 [123, 1077, 4697, 4428, 1826, 4460, 67, 200, 835, 676, 3327],
 [4898, 2930, 1020, 3347, 4551, 4269],
 [4918, 572, 2500, 1953, 3049, 1943, 3586, 4526, 374],
 [193, 3839, 863, 1906, 3230, 3255, 3961, 3020, 3882, 3200],
 [3888, 504, 2356, 1606, 1633, 3020, 3882, 3200],
 [41, 3615, 1921, 964, 1616, 16, 4443, 194, 1965, 3516],
 [357, 2334, 4694],
 [698, 4109, 2949, 983, 4551, 207, 3416, 4006],
 [496, 1138

In [17]:
sent_length=20
embed_doc=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embed_doc

array([[   0,    0,    0, ...,  791, 3599, 1269],
       [   0,    0,    0, ..., 1661, 1731, 4006],
       [   0,    0,    0, ..., 3704,  528, 1966],
       ...,
       [   0,    0,    0, ..., 3020, 3882, 3200],
       [   0,    0,    0, ..., 1340, 3530, 4777],
       [   0,    0,    0, ..., 1942, 3414, 2179]])

In [28]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [33]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Conv1D(10,3,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(LSTM(10,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 35, 40)            200000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 33, 10)            1210      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 10)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 202,061
Trainable params: 202,061
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
import numpy as np
x_final=np.array(embed_doc)
y_final=np.array(y)

In [35]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33, random_state=42)

In [36]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 10ms/step - loss: 0.5545 - accuracy: 0.6798 - val_loss: 0.3988 - val_accuracy: 0.8204
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.3086 - accuracy: 0.8853 - val_loss: 0.3450 - val_accuracy: 0.8418
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.2088 - accuracy: 0.9289 - val_loss: 0.3563 - val_accuracy: 0.8414
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1439 - accuracy: 0.9576 - val_loss: 0.3841 - val_accuracy: 0.8398
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0985 - accuracy: 0.9758 - val_loss: 0.4151 - val_accuracy: 0.8403
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0705 - accuracy: 0.9860 - val_loss: 0.4522 - val_accuracy: 0.8399
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0516 - accuracy: 0.9905 - val_loss: 0.4883 - val_accuracy: 0.8363
Epoch

In [37]:
predict_x=model.predict(x_test) 
pred=np.round(predict_x).astype(int)

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
print("Hybrid CNN-LSTM Data")
score=metrics.accuracy_score(y_test, pred)
print("accuracy: %0.3f" %score)
cm=metrics.confusion_matrix(y_test,pred)
cm

Hybrid CNN-LSTM Data
accuracy: 0.835


array([[2908,  511],
       [ 483, 2133]], dtype=int64)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.86      0.85      0.85      3419
           1       0.81      0.82      0.81      2616

    accuracy                           0.84      6035
   macro avg       0.83      0.83      0.83      6035
weighted avg       0.84      0.84      0.84      6035



In [43]:
from sklearn.metrics import precision_recall_curve
precision,recall,thres = precision_recall_curve(y_test,pred)
tpr=cm[0][0]/(cm[0][0]+cm[1][0])
fpr=cm[0][1]/(cm[0][1]+cm[1][1])
print("tpr:",tpr)
print("fpr:",fpr)

tpr: 0.8575641403715718
fpr: 0.193267776096823
